In [ ]:
!pip install folium --upgrade

In [ ]:
!pip install --upgrade pip

In [ ]:
# from datascience import *
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import folium
import json
import os
from branca.colormap import linear
import branca.colormap

## Data

We'll use the same "us-states" data from last time. For the choropleth, we'll use the "US_Unemployment_Oct2012.csv" dataset that contains unemployment rates by State for October 2012.

In [ ]:
# US States GeoJSON file
us_states = os.path.join('data', 'us-states.json')
geo_json_data = json.load(open(us_states))

# Unemployment Data
US_Unemployment_Oct2012 = os.path.join('data', 'US_Unemployment_Oct2012.csv')
unemployment = pd.read_csv(US_Unemployment_Oct2012)

## Base Map Review

Plot a basemap of the United States, overlay it with the "us_states" GeoJSON file, and customize the color scheme.

In [ ]:
usa = [39.8283, -98.5795]
m = folium.Map(
    location=usa,
    tiles='OpenStreetMap',
    zoom_start=5
)

folium.GeoJson(
    us_states,
    name='geojson',
    style_function=lambda feature: {
        'fillColor': 'white',
        'color': '#000000',
        'weight': 2,
        'dashArray': '1,1'
    }
).add_to(m)

m

## Choropleth

Choropleths are a mapping technique that are popular among data scientists because they use color to display statistical information. Generally, choropleths shade geographic regions by intensity of some measure (in our case, unemployment rates).

**Step 1**:

Before we start shading states by their unemployment rates, we need to decide on a color scale! 

Consult the branca documentation, and scroll down to "_schemes": https://pypkg.com/pypi/branca/f/branca/colormap.py

Then, create an object called colormap that uses "linear. .scale()" to create a sliding color scale that ranges from the minimum unemployment to the maximum.

In [ ]:
colormap = branca.colormap.LinearColormap(['yellow','red'],
    vmin=unemployment.Unemployment.min(),
    vmax=unemployment.Unemployment.max())

colormap

**Step 2**:

Next, we need to convert our unemployment into a dictionary, which will be passed into the folium.GeoJson argument. Take a peak at the dataframe (note that we loaded it as a pandas dataframe instead of a datascience Table) by using the "head" method. Then, create an object called "unemployment_dict" by using the "set_index" method.

In [ ]:
unemployment.head()

In [ ]:
unemployment_dict = unemployment.set_index('State')['Unemployment']
type(unemployment_dict) #this is a Pandas series; is that a problem?
unemployment_dict.head()

**Step 3**:

We're ready to plot! Use folium.Map to create a base map of the U.S., then folium.GeoJson to overlay the geo_json_data and add color. Hint: For the 'fillColor' argument, try something like this: 

'fillColor': colormap(unemployment_dict[feature['id']]

In [ ]:
# create a chloropleth map of unemployement
chloro = folium.Map(
    location=usa,
    tiles='OpenStreetMap',
    zoom_start=5
)

folium.GeoJson(
    us_states,
    name='geojson',
    style_function=lambda feature: {
        'fillColor': colormap(unemployment_dict[feature['id']]), # why does this work? feature is the index?
        'color': '#000000',
        'weight': 2,
        'dashArray': '1,1'
    }
).add_to(chloro)
folium.LayerControl().add_to(chloro) #add a layers menu to top right of map
chloro

To wrap up, let's add a color legend. Use "colormap.caption" and "colormap.add_to()" to add the legend to the map, and call it again.

In [ ]:
# revise the colormap to add caption and legend
colormap = branca.colormap.LinearColormap(['yellow','red'],
    vmin=unemployment.Unemployment.min(),
    vmax=unemployment.Unemployment.max(),
    caption='Unemployment Rate by Color')
colormap.add_to(chloro)  

chloro

#### Question 1

What patterns do you notice in the variation in state unemployment rates in 2012? What do you think might help explain some of these differences?

**Looks like the upper midwest has the lowest rates, while the more industrialized states as well as the south have higher rates. There are probably multiple explanations, like the fracking boom, recovery from recession, and the collapse of the housing market four years earlier.**

## Fun With Colors

Let's experiment with a few different options in the "colormap" module. First, try filling in the code below to customize a color pallette. Fill in three colors, and indices for the color breaks:

In [ ]:
# so we need to make a step colormap with three colors
colors = ['green', 'yellow', 'red']
colormap = branca.colormap.StepColormap(
                           colors, 
                           index=None, 
                           vmin=unemployment.Unemployment.min(),
                           vmax=unemployment.Unemployment.max(), 
                           caption='Unemployment by Category')
colormap

Now replot the map of U.S. unemployment with this new color scheme!

In [ ]:
step = folium.Map(
    location=usa,
    tiles='Stamen Toner',
    zoom_start=4
)

folium.GeoJson(
    us_states,
    name='geojson',
    style_function=lambda feature: {
        'fillColor': colormap(unemployment_dict[feature['id']]), # why does this work? feature is the index?
        'color': '#000000',
        'weight': 2,
        'dashArray': '1,1'
    }
).add_to(step)
folium.LayerControl().add_to(step) #add a layers menu to top right of map
step

Use "branca.colormap.linear" to explore different color scale gradients. Experiment with choropleths that use different color schemes!

In [ ]:
colormap = branca.colormap.LinearColormap(['violet','red'],
    vmin=unemployment.Unemployment.min(),
    vmax=unemployment.Unemployment.max(),
    caption='Unemployment Rate by Color')
colormap

In [ ]:
chloro = folium.Map(
    location=usa,
    tiles='OpenStreetMap',
    zoom_start=4
)

folium.GeoJson(
    us_states,
    name='geojson',
    style_function=lambda feature: {
        'fillColor': colormap(unemployment_dict[feature['id']]), # why does this work? feature is the index?
        'color': '#000000',
        'weight': 2,
        'dashArray': '1,1'
    }
).add_to(chloro)
folium.LayerControl().add_to(chloro) #add a layers menu to top right of map
chloro

Finally, let's create a map that allows you to highlight states and zoom in on them. Use the "highlight" argument to add this feature.

In [ ]:
# this cell is self-contained and does not call any variables defined above except us_states
# it reads csv file into new dataframe 
# it also uses the chloropleth method in folium, rather than add it as style function definition
# but the chloropleth method soon to be deprecated

state_data = pd.read_csv(US_Unemployment_Oct2012)

m = folium.Map(location=usa, zoom_start=4)
m.choropleth(
    geo_data=us_states,
    data=state_data,
    columns=['State', 'Unemployment'],
    key_on='feature.id',
    fill_color='YlGn',
    fill_opacity=0.8,
    line_opacity=0.2,
    legend_name='Unemployment Rate (%)',
    highlight=True
)

m

In [ ]:
# same as above with current definition of choropleth as a class rather than a method
state_data = pd.read_csv(US_Unemployment_Oct2012)

m = folium.Map(location=usa, zoom_start=4)
folium.Choropleth(
    geo_data=us_states,
    data=state_data,
    columns=['State', 'Unemployment'],
    key_on='feature.id',
    fill_color='YlGn',
    fill_opacity=0.8,
    line_opacity=0.2,
    legend_name='Unemployment Rate (%)',
    highlight=True
).add_to(m)

m